<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

nlp = spacy.load("en_core_web_lg")

df = yelp[['stars', 'text']]

def tokenize(document):
    words = [""]
    
    STOP_WORDS = nlp.Defaults.stop_words.union(words)
    
    doc = nlp(document)

    lemmas = []
    
    for token in doc:
        if (token.text.lower() not in STOP_WORDS) and ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
            lemmas.append(token.lemma_)

    return lemmas

df['tokens'] = df['text'].apply(tokenize)

C:\ProgramData\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
df.head()

,stars,text,tokens
0,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...","[beware, fake, fake, fake, small, business, Lo..."
1,4,Came here for lunch Togo. Service was quick. S...,"[come, lunch, Togo, service, quick, staff, fri..."
2,3,I've been to Vegas dozens of times and had nev...,"[Vegas, dozen, time, step, foot, Circus, Circu..."
3,1,We went here on a night where they closed off ...,"[go, night, close, street, party, good, actual..."
4,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...","[3.5, 4, star, \n\n, bad, price, $, 12.99, lun..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [5]:
review_text_tokens = []
for set_of_tokens in df['tokens'].values:
    working_set = ""
    for token in set_of_tokens:
        working_set += token + ' '
    review_text_tokens.append(working_set)

In [35]:
review_text_tokens[0][:10]

'beware fak'

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate vectorizer object
count_vect = CountVectorizer(tokenizer=tokenize, max_df=.98, min_df=3, ngram_range=(1,2))
count_vect.fit(review_text_tokens)
dtm_count = count_vect.transform(review_text_tokens)
dtm_count = pd.DataFrame(dtm_count.todense(), columns=count_vect.get_feature_names())
dtm_count.head()

,,,,,,,,,$,1,...,à,à ce,à essayer,à la,ça,équipe,était,été,être,‍
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.98, ngram_range=(1,2))
dtm_tfidf = tfidf.fit_transform(review_text_tokens)
dtm_tfidf = pd.DataFrame(dtm_tfidf.todense(), columns=tfidf.get_feature_names())
dtm_tfidf.head()

,,,,,,$,1,10,15,2,...,week,weekend,wife,wine,wish,wonderful,work,worth,wrong,year
0,0.000000,0.0,0.0,0.401464,0.0,0.145458,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.068850,0.0,0.0,0.130594,0.0,0.189266,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108949
3,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.362818,0.0,0.0,0.172046,0.0,0.124671,0.0,0.0,0.187231,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [38]:
from sklearn.neighbors import NearestNeighbors

nn_count = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn_tfidf = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')

nn_count.fit(dtm_count)
nn_tfidf.fit(dtm_tfidf)


NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [39]:
sample_yelp_review = ["Love this place!  The set up makes for a very romantic date spot with it's warm and cozy feel.  I enjoyed all of the vintage photos on the walls.  We will be making Royal Oak one of our regular stops.  Don't forget to check out the second floor!"]

In [40]:
new_count = count_vect.transform(sample_yelp_review)
new_tfidf = tfidf.transform(sample_yelp_review)

In [41]:
nn_count.kneighbors(new_count.todense())

(array([[5.38516481, 5.47722558, 5.47722558, 5.56776436, 5.65685425]]),
 array([[6311, 5129, 6875, 3693, 2515]], dtype=int64))

In [44]:
df.text[5129]

'Best place everrrrr!!!'

In [45]:
nn_tfidf.kneighbors(new_tfidf.todense())

(array([[1.        , 1.        , 1.        , 1.11958655, 1.12431662]]),
 array([[6311, 4839, 8413, 8006,  778]], dtype=int64))

In [46]:
df.text[8006]

"Bomb low key spot...my bf and I loved it!! It's like my momma was here making this! #posoleshawwty"

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [56]:
# nlp = spacy.load("en_core_web_lg")

In [19]:
# def tokenize(document):
#     doc = nlp(document)
#     return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [49]:
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

clf = XGBClassifier()

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

# count_vect = CountVectorizer(tokenizer=tokenize, max_df=.98, min_df=3, ngram_range=(1,2), error_score='raise')

# tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.98, ngram_range=(1,2))



lsi_count = Pipeline([
                 #Vectorizer
                 ('vect', count_vect), 
                 # Classifier
                 #('svd', svd)
                ])
lsi_tfidf = Pipeline([
                 #Vectorizer
                 ('vect', tfidf), 
                 # Classifier
                 ('svd', svd)
                ])

pipe_count = Pipeline([('lsi', lsi_count), ('clf', clf)])
pipe_tfidf = Pipeline([('lsi', lsi_tfidf), ('clf', clf)])

In [51]:
from sklearn.ensemble import RandomForestClassifier

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect),
                 ('clf', clf)])

In [54]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(df.text, df.stars)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:   29.6s finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [57]:
grid_search.best_score_

0.5385

In [56]:
grid_search.predict(sample_yelp_review)

array([5], dtype=int64)

In [64]:
grid_search.fit(review_text_tokens, df.stars)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:   23.2s finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [65]:
grid_search.best_score_

0.5368

In [50]:
parameters = {
    #'lsi__svd__n_components': [250],
    'lsi__vect__max_df': [.95, .98],
    #'clf__max_depth':[5, 10],
    #'clf__tree_method': ['gpu_hist'],
    #'clf__num_leaves': (6),
    #'clf__min_child_samples': (100),
    #'clf__min_child_weight': [1e-5, 1]
}

grid_search_count = GridSearchCV(pipe_count,parameters, cv=3, n_jobs=4, verbose=1)
grid_search_count.fit(review_text_tokens, df.stars)
#grid_search_count.best_score_

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:   17.1s remaining:    0.0s


OSError: [E050] Can't find model 'en_model.vectors'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
grid_search_count.predict(sample_yelp_review)

In [9]:
parameters = {
    'lsi__svd__n_components': [250],
    'lsi__vect__max_df': [.95, .98],
    #'clf__max_depth':[5, 10],
    #'clf__tree_method': ['gpu_hist'],
    #'clf__num_leaves': (6),
    #'clf__min_child_samples': (100),
    #'clf__min_child_weight': [1e-5, 1]
}

grid_search_tfidf = GridSearchCV(pipe_tfidf,parameters, cv=3, n_jobs=8, verbose=1)
grid_search_tfidf.fit(review_text_tokens, df.stars)
grid_search_tfidf.best_score_

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


OSError: [E050] Can't find model 'en_model.vectors'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
grid_search_tfidf.predict(sample_yelp_review)

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [21]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from gensim import corpora

Learn the vocubalary of the yelp data:

In [22]:
id2word = corpora.Dictionary(df.tokens)

Create a bag of words representation of the entire corpus

In [23]:
corpus = [id2word.doc2bow(text) for text in df.tokens]

Your LDA model should be ready for estimation: 

In [25]:
import warnings
warnings.filterwarnings('ignore')

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=8,
                   num_topics = 10 # You can change this parameter
                  )

In [26]:
len(id2word.keys())

28661

In [27]:
id2word.filter_extremes(no_below=20, no_above=0.98)

In [28]:
len(id2word.keys())

2546

Create 1-2 visualizations of the results

In [16]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\ProgramData\Anaconda3\envs\U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.003778  0.000700       1        1  10.733779
0      0.003480 -0.000765       2        1  10.495767
5     -0.000372  0.004705       3        1  10.406908
1      0.002419 -0.001157       4        1  10.275527
6      0.002005 -0.002763       5        1   9.915990
9     -0.001318  0.002090       6        1   9.796505
4     -0.000185 -0.001831       7        1   9.787773
8     -0.001760  0.001565       8        1   9.771530
3     -0.003202  0.001789       9        1   9.532938
2     -0.004844 -0.004335      10        1   9.283280, topic_info=     Category          Freq        Term         Total  loglift  logprob
0     Default  13764.000000              13764.000000  30.0000  30.0000
146   Default   4687.000000        food   4687.000000  29.0000  29.0000
43    Default   3756.000000        come   3756.000000  28.0000  28.0000
82    Default   2792.000000         get   2792.000000  27.0000  27.0000
55    Default   8150.000000        \n\n   8150.000000  26.0000  26.0000
374   Default   3291.000000       order   3291.000000  25.0000  25.0000
321   Default   3053.000000          \n   3053.000000  24.0000  24.0000
41    Default   1623.000000        wait   1623.000000  23.0000  23.0000
24    Default   3634.000000        like   3634.000000  22.0000  22.0000
131   Default   5070.000000       place   5070.000000  21.0000  21.0000
51    Default   3449.000000     service   3449.000000  20.0000  20.0000
1     Default   2165.000000           $   2165.000000  19.0000  19.0000
39    Default   4123.000000        time   4123.000000  18.0000  18.0000
35    Default   1486.000000         say   1486.000000  17.0000  17.0000
356   Default   1540.000000        find   1540.000000  16.0000  16.0000
46    Default   6134.000000        good   6134.000000  15.0000  15.0000
208   Default   4126.000000       great   4126.000000  14.0000  14.0000
106   Default   1615.000000  restaurant   1615.000000  13.0000  13.0000
184   Default   1370.000000   recommend   1370.000000  12.0000  12.0000
156   Default   1690.000000       price   1690.000000  11.0000  11.0000
266   Default   1411.000000  experience   1411.000000  10.0000  10.0000
298   Default   1140.000000       thing   1140.000000   9.0000   9.0000
457   Default   1630.000000        know   1630.000000   8.0000   8.0000
83    Default   2964.000000          go   2964.000000   7.0000   7.0000
443   Default    977.000000        give    977.000000   6.0000   6.0000
429   Default   1071.000000    customer   1071.000000   5.0000   5.0000
1193  Default   1162.000000     chicken   1162.000000   4.0000   4.0000
344   Default   1395.000000  definitely   1395.000000   3.0000   3.0000
301   Default   1751.000000        want   1751.000000   2.0000   2.0000
144   Default   1490.000000         eat   1490.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
184   Topic10    170.730637   recommend   1370.924927   0.2938  -5.6460
24    Topic10    407.788849        like   3634.631592   0.1894  -4.7753
704   Topic10    136.501343       taste   1104.676758   0.2860  -5.8698
329   Topic10     91.754219         big    715.567322   0.3230  -6.2670
164   Topic10    169.697144       think   1421.325806   0.2516  -5.6521
358   Topic10    110.928665       fresh    911.310852   0.2710  -6.0772
46    Topic10    593.947144        good   6134.181641   0.0421  -4.3993
345   Topic10    137.726379   delicious   1177.227783   0.2313  -5.8608
144   Topic10    166.312256         eat   1490.761597   0.1838  -5.6722
39    Topic10    399.739716        time   4123.773438   0.0432  -4.7953
374   Topic10    328.618439       order   3291.409912   0.0728  -4.9912
55    Topic10    713.097107        \n\n   8150.744629  -0.0593  -4.2165
234   Topic10    125.882874         way   1104.592651   0.2051  -5.9507
430   Topic10    158.359726         day   1458.562866   0.1566  -5.72

In [18]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, df_column, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    warnings.filterwarnings('ignore')
    
    coherence_values = []
    
    tokens = list(df_column)
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            stream = df_column
            model = LdaMulticore(corpus=corpus,
                                 num_topics=num_topics,
                                 id2word=dictionary,
                                 random_state=723812,
                                 workers=8)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [30]:
import warnings
warnings.filterwarnings('ignore')

coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, 
                                                        df_column=df.tokens, 
                                                        start=2, 
                                                        limit=40, 
                                                        step=6,
                                                        passes=2)

KeyboardInterrupt: 

In [31]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

NameError: name 'coherence_values' is not defined

In [32]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

NameError: name 'topic_coherence' is not defined

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)